In [1]:
import pandas as pd
import numpy as np

In [4]:
tips = pd.read_csv('tips.csv')
tips['tip_pct'] = tips['tip'] / tips['total_bill']

### suppose you wanted to select the top five tip_pct values by group.

In [5]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]
top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


Now, if we group by smoker, say, and call apply with this function, we get the
following:

In [6]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

What has happened here? The top function is called on each row group from the
DataFrame, and then the results are glued together using pandas.concat, labeling the
pieces with the group names. The result therefore has a hierarchical index whose
inner level contains index values from the original DataFrame.

In [7]:
# If you pass a function to apply that takes other arguments or keywords, you can pass these after the function:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

# Beyond these basic usage mechanics, getting the most out of apply 
# may require some creativity. What occurs inside the function
# passed is up to you; it only needs to return a pandas object or a
# scalar value. The rest of this chapter will mainly consist of examples
# showing you how to solve various problems using groupby.

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

In [9]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [10]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

Inside GroupBy, when you invoke a method like describe, it is actually just a short‐
cut for:

f = lambda x: x.describe()

grouped.apply(f)

### Suppressing the Group Keys

In the preceding examples, you see that the resulting object has a hierarchical index
formed from the group keys along with the indexes of each piece of the original
object. You can disable this by passing group_keys=False to groupby:

In [11]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


### Quantile and Bucket Analysis

pandas has some tools, in particular cut and qcut,
for slicing data up into buckets with bins of your choosing or by sample quantiles.
Combining these functions with groupby makes it convenient to perform bucket or
quantile analysis on a dataset. Consider a simple random dataset and an equal-length
bucket categorization using cut:

In [12]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

quartiles = pd.cut(frame.data1, 4)
quartiles[:10]

0    (-0.239, 1.493]
1    (-1.97, -0.239]
2    (-1.97, -0.239]
3    (-1.97, -0.239]
4    (-1.97, -0.239]
5    (-0.239, 1.493]
6     (1.493, 3.224]
7    (-1.97, -0.239]
8    (-1.97, -0.239]
9     (1.493, 3.224]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.709, -1.97] < (-1.97, -0.239] < (-0.239, 1.493] < (1.493, 3.224]]

The Categorical object returned by cut can be passed directly to groupby. So we
could compute a set of statistics for the data2 column like so:

In [13]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quartiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.709, -1.97]",-1.918271,1.583347,24.0,-0.016227
"(-1.97, -0.239]",-2.781699,2.987020,380.0,0.061513
"(-0.239, 1.493]",-2.786964,3.468787,524.0,0.014327
"(1.493, 3.224]",-2.773301,2.729348,72.0,-0.055488


These were equal-length buckets; to compute equal-size buckets based on sample
quantiles, use qcut. I’ll pass labels=False to just get quantile numbers:

In [14]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.639822,2.709366,100.0,-0.036000
1,-2.255251,2.091676,100.0,0.095757
2,-2.324812,2.987020,100.0,0.278701
3,-2.781699,2.224674,100.0,-0.110528
4,-2.786964,2.624641,100.0,0.007372
5,-2.592713,2.489259,100.0,-0.013578
6,-2.631123,2.229802,100.0,0.004461
7,-2.461812,3.468787,100.0,0.010308
8,-2.354838,2.778273,100.0,0.046978


### Example: Filling Missing Values with Group-Specific Values

When cleaning up missing data, in some cases you will replace data observations
using dropna, but in others you may want to impute (fill in) the null (NA) values
using a fixed value or some value derived from the data. fillna is the right tool to
use; for example, here I fill in NA values with the mean:

In [15]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -1.441713
2         NaN
3   -0.148615
4         NaN
5   -0.107349
dtype: float64

In [16]:
s.fillna(s.mean())

0   -0.565892
1   -1.441713
2   -0.565892
3   -0.148615
4   -0.565892
5   -0.107349
dtype: float64

Suppose you need the fill value to vary by group. One way to do this is to group the
data and use apply with a function that calls fillna on each data chunk. Here is
some sample data on US states divided into eastern and western regions:

In [17]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data

# Note that the syntax ['East'] * 4 produces a list containing four copies of the 
# elements in ['East']. Adding lists together concatenates them.

Ohio         -0.081712
New York     -0.808217
Vermont       1.664378
Florida       0.448712
Oregon       -2.248408
Nevada        1.947742
California   -1.358093
Idaho        -0.551634
dtype: float64

In [20]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data
group_key

['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

In [19]:
data.groupby(group_key).mean()

East   -0.147072
West   -1.803250
dtype: float64

We can fill the NA values using the group means like so:

In [21]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

East  Ohio         -0.081712
      New York     -0.808217
      Vermont      -0.147072
      Florida       0.448712
West  Oregon       -2.248408
      Nevada       -1.803250
      California   -1.358093
      Idaho        -1.803250
dtype: float64

In another case, you might have predefined fill values in your code that vary by
group. Since the groups have a name attribute set internally, we can use that:

In [22]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

East  Ohio         -0.081712
      New York     -0.808217
      Vermont       0.500000
      Florida       0.448712
West  Oregon       -2.248408
      Nevada       -1.000000
      California   -1.358093
      Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

Suppose you wanted to draw a random sample (with or without replacement) from a
large dataset for Monte Carlo simulation purposes or some other application. There
are a number of ways to perform the “draws”; here we use the sample method for
Series.

In [23]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)

So now we have a Series of length 52 whose index contains card names and values are
the ones used in Blackjack and other games (to keep things simple, I just let the ace
'A' be 1):

In [24]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

Now, based on what I said before, drawing a hand of five cards from the deck could
be written as:

In [25]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

9C     9
6C     6
3H     3
QS    10
8H     8
dtype: int64

Suppose you wanted two random cards from each suit. Because the suit is the last
character of each card name, we can group based on this and use apply:

In [26]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  7C      7
   AC      1
D  10D    10
   9D      9
H  4H      4
   7H      7
S  2S      2
   6S      6
dtype: int64

In [27]:
# Alternatively, we could write:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

8C      8
5C      5
AD      1
8D      8
10H    10
6H      6
AS      1
9S      9
dtype: int64

### Example: Group Weighted Average and Correlation

Under the split-apply-combine paradigm of groupby, operations between columns in
a DataFrame or two Series, such as a group weighted average, are possible. As an
example, take this dataset containing group keys, values, and some weights:

In [28]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                                'data': np.random.randn(8),
                                'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.225511,0.966192
1,a,-0.874757,0.076341
2,a,0.431431,0.287035
3,a,2.499620,0.503673
4,b,-1.216473,0.620132
5,b,0.119011,0.217392
6,b,-0.502814,0.653765
7,b,-0.344911,0.485670


The group weighted average by category would then be:

In [29]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.836733
b   -0.619505
dtype: float64

As another example, consider a financial dataset originally obtained from Yahoo!
Finance containing end-of-day prices for a few stocks and the S&P 500 index (the SPX
symbol):

In [31]:
close_px = pd.read_csv('stock_px.csv', parse_dates=True,index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [32]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


One task of interest might be to compute a DataFrame consisting of the yearly corre‐
lations of daily returns (computed from percent changes) with SPX. As one way to do
this, we first create a function that computes the pairwise correlation of each column
with the 'SPX' column:

In [33]:
spx_corr = lambda x: x.corrwith(x['SPX'])

In [34]:
# Next, we compute percent change on close_px using pct_change:
rets = close_px.pct_change().dropna()

Lastly, we group these percent changes by year, which can be extracted from each row
label with a one-line function that returns the year attribute of each datetime label:

In [35]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


You could also compute inter-column correlations. Here we compute the annual cor‐
relation between Apple and Microsoft:

In [36]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### Example: Group-Wise Linear Regression

In the same theme as the previous example, you can use groupby to perform more
complex group-wise statistical analysis, as long as the function returns a pandas
object or scalar value. For example, I can define the following regress function
(using the statsmodels econometrics library), which executes an ordinary least
squares (OLS) regression on each chunk of data:

In [37]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [38]:
# Now, to run a yearly linear regression of AAPL on SPX returns, execute:
by_year.apply(regress, 'AAPL', ['SPX'])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514
